In [36]:
import cv2
import numpy as np
import pandas as pd
import argparse
import os
import re
import matplotlib.pyplot as plt
from natsort import natsorted 
from scipy.fft import fft, fftfreq
from numpy import diff, sqrt, arange, ceil, log2
from scipy.ndimage import gaussian_filter1d
    
try:
    from tqdm import tqdm
except ImportError:
    print("`pip install tqdm` to have a progress bar")
    def tqdm(iterable, *args, **kwargs):
        for i, item in enumerate(iterable, 1):
            print(f"Processing item {i}/{len(iterable)}")
            yield item           
            
video_dir = "/Users/sienajobson/Desktop/MSc Neuroaesthetics/research project/videos30/"

file_list = natsorted([f for f in os.listdir(video_dir) if f.endswith(".mp4")])

for filename in file_list:
    file_path = os.path.join(video_dir, filename)
    parser = argparse.ArgumentParser()
    parser.add_argument("--video", type=str, help="video file path", default=file_path)
    parser.add_argument("--frame_thresh", type=int, help="number of 'blank' frames before a new stroke", default=5)
    parser.add_argument("--distance_thresh", type=float, help="distance thresh for a new stroke", default=30)
    parser.add_argument("--area_thresh", type=float, help="area thresh for new point", default=0.5)
    parser.add_argument("-f", type=str, help="Dummy argument for Jupyter compatibility", default="")
    cfg = parser.parse_args()

    cap = cv2.VideoCapture(cfg.video)
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = cap.get(cv2.CAP_PROP_FPS)

    # Hacky box for cropping  (approximate!)
    l, t, r, b = 0, 20, 2000, 1800

    def read_frame():
        # Crops to the area we are interested in
        ret, frame = cap.read()
        if ret:
            frame = frame[t:b, l:r, :]
        return ret, frame

    # Read the first frame
    ret, prev_frame = read_frame()
    if not ret:
        print("Error: Unable to read video")
        cap.release()
        raise(ValueError)

    base_name = '.'.join(cfg.video.split('.')[:-1])
    csv_file = base_name + '.csv'
    f = open(csv_file, "w")
    f.write("Frame,X,Y\n")

    ind = 0
    strokes = [[]]
    lastp = np.zeros(2)
    pause_count = 0

    x_positions = []
    y_positions = []
    recent_velocities = []

    for i in tqdm(range(num_frames), desc="Processing frames"):
        ret, frame = read_frame()
        if not ret:
            break

        frame_diff = cv2.absdiff(prev_frame, frame)
        gray_diff = cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray_diff, 30, 255, cv2.THRESH_BINARY)

        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if not contours:
            pause_count += 1
        else:
            pause_count = 0

            for contour in contours:
                area = cv2.contourArea(contour)
                if area > 0.2:  # Lowered threshold
                    M = cv2.moments(contour)

        # Adaptive thresholds based on recent velocities
        avg_velocity = np.mean(recent_velocities) if recent_velocities else 0
        adaptive_frame_thresh = cfg.frame_thresh + int(avg_velocity / 10)
        adaptive_dist_thresh = cfg.distance_thresh + avg_velocity * 0.5

        if pause_count > adaptive_frame_thresh:
            if strokes[-1]:
                strokes.append([])

        for contour in contours:
            new_pts = []
            if cv2.contourArea(contour) > cfg.area_thresh:
                M = cv2.moments(contour)
                if M["m00"] != 0:
                    cx = M["m10"] / M["m00"]
                    cy = M["m01"] / M["m00"]
                    new_pts.append(np.array((cx, cy)))

                    x_positions.append(cx)
                    y_positions.append(cy)
                    f.write(f"{int(cap.get(cv2.CAP_PROP_POS_FRAMES))},{int(cx)},{int(cy)}\n")

            if new_pts:
                dists = [np.linalg.norm(p - lastp) for p in new_pts]
                newp = new_pts[np.argmin(dists)]
                velocity = np.linalg.norm(newp - lastp)
                recent_velocities.append(velocity)
                if len(recent_velocities) > 5:
                    recent_velocities.pop(0)

                if (velocity > adaptive_dist_thresh and strokes[-1]):
                    strokes.append([])
                lastp = newp
                strokes[-1].append(lastp)

        prev_frame = frame

    f.close()
    strokes = [np.array(P) for P in strokes if len(P) > 1]

    print(f"\nPosition data saved to {csv_file}")
    cap.release()

    # plt.figure(figsize=(5,5))
    # for P in strokes[:]:
    #     plt.plot(P[:,0], P[:,1])
    # plt.axis('equal')
    # plt.gca().invert_yaxis()
    # plt.savefig(base_name + '.pdf')
    # plt.show()

    ########### COMPUTING FEATURES ############
    sigma =1

    # alternative to SG!
    def gauss_derivative(X, order, sigma):
        return gaussian_filter1d(X, sigma, order=order, axis=0)

    def compute_stroke_length(X):
        deltas = np.diff(X, axis=0)  # Differences between consecutive points
        distances = np.sqrt((deltas ** 2).sum(axis=1))  # Euclidean distances
        return distances.sum()

    def compute_speed(X, sigma=1, sampling_rate=frame_rate):
        dX = gauss_derivative(X, 1, sigma)
        speed = np.sqrt(dX[:, 0]**2 + dX[:, 1]**2) * sampling_rate
        return speed

    def compute_acceleration(X, sigma=1, sampling_rate=frame_rate):
        d2X = gauss_derivative(X, 2, sigma)
        acceleration = np.sqrt(d2X[:, 0]**2 + d2X[:, 1]**2) * (sampling_rate ** 2)
        return acceleration

    def compute_jerk(X, sigma=1, sampling_rate=frame_rate):
        d3X = gauss_derivative(X, 3, sigma)
        jerk = np.sqrt(d3X[:, 0]**2 + d3X[:, 1]**2) * (sampling_rate ** 3)
        return jerk

    def compute_dimensionless_jerk(X, sampling_rate=frame_rate):
        jerk = compute_jerk(X, sigma=1) * (sampling_rate ** 3)  # Convert to units/sec³
        jerk_squared_integral = np.sum(jerk ** 2) / sampling_rate  # ∫ jerk² dt
        duration = len(X) / sampling_rate  # T
        total_distance = np.sum(compute_speed(X)) / sampling_rate  # A ≈ path length

        if total_distance == 0 or duration == 0:
            return np.nan

        dless_jerk = (jerk_squared_integral * duration**5) / (total_distance**2)
        return dless_jerk

    
    # e.g., in Flash & Hogan 1985, or Teulings 1997
    # Parkinsonism reduces coordination of fingers, wrist, and arm in fine motor control
    # Sensitivity of smoothness measures to movement duration, amplitude, and arrests
    # On the analysis of movement smoothness

    def compute_sparc(speed, sampling_rate=frame_rate, padlevel=4, cutoff=10.0, amp_th=0.05):
        """
        Parameters:
            speed : np.ndarray 
            sampling_rate : float
                Sampling frequency in Hz (default: 30.0)
            padlevel : int
                Zero-padding level for FFT (default: 4)
            cutoff : float
                Cutoff frequency for spectrum selection (default: 10.0)
            amp_th : float
                Amplitude threshold to define the selected frequency region (default: 0.05)
        Returns:
            float: SPARC value (lower = less smooth)
        """
        if len(speed) < 5:
            return np.nan

        # zero-padding for FFT
        nfft = int(2 ** (ceil(log2(len(speed))) + padlevel))

        # frequency array and magnitude spectrum
        f = arange(0, sampling_rate, sampling_rate / nfft)
        Mf = np.abs(fft(speed, nfft))
        Mf = Mf[:len(f)]
        if Mf.max() == 0:
            return np.nan
        Mf = Mf / Mf.max()  # normalize

        # select spectrum below cutoff freq
        f_sel = f[f <= cutoff]
        Mf_sel = Mf[:len(f_sel)]

        # apply amplitude threshold
        above_thresh = np.where(Mf_sel >= amp_th)[0]
        if len(above_thresh) < 2:
            return np.nan
        f_sel = f_sel[above_thresh[0]:above_thresh[-1] + 1]
        Mf_sel = Mf_sel[above_thresh[0]:above_thresh[-1] + 1]

        # compute spectral arc length (negative)
        norm_freq = np.diff(f_sel) / (f_sel[-1] - f_sel[0])
        sparc = -np.sum(np.sqrt(norm_freq**2 + np.diff(Mf_sel)**2))

        return sparc

    def extract_features(strokes, sampling_rate=frame_rate):
        strokes_duration = []
        strokes_length = []
        speed_data = []
        acceleration_data = []
        jerk_data = []
        dimensionless_jerk_data = []
        sparc_scores = []
        ind = 0  # Time index tracking
        for X in strokes:
            duration = len(X)/sampling_rate
            length = compute_stroke_length(X)
            speed = compute_speed(X)
            acceleration = compute_acceleration(X)
            jerk = compute_jerk(X)
            norm_dless_jerk = compute_dimensionless_jerk(X, sampling_rate)
            sparc = compute_sparc(speed, sampling_rate)

            strokes_duration.append(duration)
            strokes_length.append(length)

            # Store results with corresponding time index
            n = len(speed)
            time = np.linspace(ind, ind + n - 1, n)
            speed_data.append((time, speed))

            n = len(acceleration)
            time = np.linspace(ind, ind + n - 1, n)
            acceleration_data.append((time, acceleration))

            n = len(jerk)
            time = np.linspace(ind, ind + n - 1, n)
            jerk_data.append((time, jerk))

            dimensionless_jerk_data.append((ind, norm_dless_jerk))

            sparc_scores.append(sparc)

            ind += len(X)

        return strokes_duration, strokes_length, speed_data, acceleration_data, jerk_data, dimensionless_jerk_data, sparc_scores

    def extract_features_means_SDs(strokes, sampling_rate=frame_rate):
        strokes_duration = []
        strokes_length = []
        speed_stats = []
        acceleration_stats = []
        jerk_stats = []
        dimensionless_jerk_data = []
        sparc_scores = []
        ind = 0  # Time index tracking
        for X in strokes:
            duration = len(X)/sampling_rate
            length = compute_stroke_length(X)
            speed = compute_speed(X)
            acceleration = compute_acceleration(X)
            jerk = compute_jerk(X)
            norm_dless_jerk = compute_dimensionless_jerk(X, sampling_rate)
            sparc = compute_sparc(speed, sampling_rate)

            strokes_duration.append(duration)
            strokes_length.append(length)        

            speed_mean = np.mean(speed)
            speed_std = np.std(speed)

            acceleration_mean = np.mean(acceleration)
            acceleration_std = np.std(acceleration)

            jerk_mean = np.mean(jerk)
            jerk_std = np.std(jerk)

            # Append a tuple of (mean, std) for each characteristic
            speed_stats.append((speed_mean, speed_std))
            acceleration_stats.append((acceleration_mean, acceleration_std))
            jerk_stats.append((jerk_mean, jerk_std))

            dimensionless_jerk_data.append((norm_dless_jerk))

            sparc_scores.append(sparc)

            ind += len(X)
        return strokes_duration, strokes_length, speed_stats, acceleration_stats, jerk_stats, dimensionless_jerk_data,  sparc_scores

    def plot_feature(data, title, ylabel):
        plt.figure(figsize=(30, 3))
        for t, values in data:
            plt.plot(t, values)
        plt.title(title)
        plt.xlabel("Time")
        plt.ylabel(ylabel)
        plt.grid(True)

    def plot_all_features(strokes):
        strokes_duration, strokes_length, speed_data, acceleration_data, jerk_data, dimensionless_jerk_data, sparc_scores = extract_features(strokes)
        plot_feature(speed_data, "Speed of Strokes", "Speed (pixels/frame)")
        plot_feature(acceleration_data, "Acceleration of Strokes", "Acceleration (pixels/frame²)")
        plot_feature(jerk_data, "Jerk of Strokes", "Jerk (pixels/frame³)")
        plt.show()
        print("\nSPARC Smoothness Scores (per stroke):")
        #for i, score in enumerate(sparc_scores):        

    #plot_all_features(strokes)
    ########################

    def parse_filename(filename):
        base = os.path.basename(filename)
        name, _ = os.path.splitext(base)
        parts = name.split('_')
        participant_id = parts[0]
        diagnosis = parts[2]
        e_order = parts[3]
        emotion = parts[4]
        code = parts[5]
        return participant_id, diagnosis, e_order, emotion, code

    def append_master(strokes, filename, master_csv_path='second_pass.csv', sampling_rate=frame_rate):
        # Extract features
        (
            strokes_duration,
            strokes_length,
            speed_stats,
            acceleration_stats,
            jerk_stats,
            dimensionless_jerk_data,
            sparc_scores
        ) = extract_features_means_SDs(strokes, sampling_rate=frame_rate)

        # Extract metadata
        participant_id, diagnosis, e_order, emotion, code = parse_filename(filename)

        # Create per-stroke summary rows
        summary_rows = []
        for i in range(len(strokes)):
            row = {
                "ParticipantID": participant_id,
                "Diagnosis": diagnosis,
                "E/Order": e_order,
                "Emotion": emotion,
                "Quality": code,
                "Stroke": i + 1,
                "Stroke Duration (seconds)": strokes_duration[i],
                "Stroke Length (pixels)": strokes_length[i],
                "Speed Mean": speed_stats[i][0],
                "Speed SD": speed_stats[i][1],
                "Accel Mean": acceleration_stats[i][0],
                "Accel SD": acceleration_stats[i][1],
                "Jerk Mean": jerk_stats[i][0],
                "Jerk SD": jerk_stats[i][1],
                "dlessJerk": dimensionless_jerk_data[i],
                "SPARC": sparc_scores[i]
            }
            summary_rows.append(row)

        df = pd.DataFrame(summary_rows)

        if not os.path.exists(master_csv_path):
            df.to_csv(master_csv_path, index=False)
            print(f" Created new master file: {master_csv_path}")
        else:
            df.to_csv(master_csv_path, mode='a', header=False, index=False)
            print(f" Appended data to existing master file: {master_csv_path}")

    append_master(strokes, filename)



Processing frames: 100%|█████████▉| 1470/1471 [00:11<00:00, 129.36it/s]



Position data saved to /Users/sienajobson/Desktop/MSc Neuroaesthetics/research project/videos30/IEW10191_drawing_2_e1_1_Y_30.csv
 Appended data to existing master file: second_pass.csv


Processing frames: 100%|█████████▉| 1470/1471 [00:13<00:00, 108.71it/s]



Position data saved to /Users/sienajobson/Desktop/MSc Neuroaesthetics/research project/videos30/IEW10191_drawing_2_e2_4_Y_30.csv
 Appended data to existing master file: second_pass.csv


Processing frames: 100%|█████████▉| 1470/1471 [00:11<00:00, 127.39it/s]


...


Position data saved to /Users/sienajobson/Desktop/MSc Neuroaesthetics/research project/videos30/ZNA22031_drawing_2_e2_4_R_30.csv
 Appended data to existing master file: second_pass.csv


Processing frames: 100%|█████████▉| 1740/1741 [00:14<00:00, 119.80it/s]



Position data saved to /Users/sienajobson/Desktop/MSc Neuroaesthetics/research project/videos30/ZNA22031_drawing_2_e3_2_G_30.csv
 Appended data to existing master file: second_pass.csv


Processing frames: 100%|█████████▉| 1740/1741 [00:12<00:00, 137.07it/s]


Position data saved to /Users/sienajobson/Desktop/MSc Neuroaesthetics/research project/videos30/ZNA22031_drawing_2_e4_3_G_30.csv
 Appended data to existing master file: second_pass.csv


In [2]:
pip install natsort


Note: you may need to restart the kernel to use updated packages.


In [7]:
print(frame_rate)

37.8644715947759


In [28]:
import pandas as pd

# Load your CSV
df = pd.read_csv("second_pass.csv")

# Group by Participant and Emotion (i.e., each video), then average motion features
video_summary = df.groupby(['ParticipantID', 'Emotion'], as_index=False).agg({
    'Stroke Duration (seconds)': 'mean',
    'Stroke Length (pixels)': 'mean',
    'Speed Mean': 'mean',
    'Speed SD': 'mean',
    'Accel Mean': 'mean',
    'Accel SD': 'mean',
    'Jerk Mean': 'mean',
    'Jerk SD': 'mean',
    'dlessJerk': 'mean',
    'SPARC': 'mean',
    # Add other motion features here
})

# If Diagnosis is participant-level, merge it back in
diagnosis = df[['ParticipantID', 'Diagnosis']].drop_duplicates()

video_summary = pd.merge(video_summary, diagnosis, on='ParticipantID', how='left')


In [30]:
# Save to CSV (no index column in the file)
video_summary.to_csv("video_summary.csv", index=False)
